# 4장 모델 훈련

## 기본 설정

- 필수 모듈 불러오기
- 그래프 출력 관련 기본 설정 지정

In [ ]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)


In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

## 과제 1

조기 종료를 사용한 배치 경사 하강법으로 로지스틱 회귀를 구현하라.
단, 사이킷런을 전혀 사용하지 않아야 한다.

기본 설정

### __1. 데이터 준비__

In [ ]:
X = iris["data"][:, 2:]  #꽃잎 길이
y = (iris["target"]==0).astype(np.int) #세토사를 타겟으로 설정

편향에 특성 추가

In [ ]:
X_with_bias = np.c_[np.ones([len(X), 1]), X]

결과를 일정하게 유지하기 위해 랜덤 시드를 지정한다.

In [ ]:
np.random.seed(2042)

### __단계 2: 데이터셋 분할__ 

데이터셋을 훈련, 검증, 테스트 용도로 6대 2대 2의 비율로 무작위로 분할한다.

In [ ]:
test_ratio = 0.2                                         # 테스트 세트 비율 = 20%
validation_ratio = 0.2                                   # 검증 세트 비율 = 20%
total_size = len(X_with_bias)                            # 전체 데이터셋 크기

test_size = int(total_size * test_ratio)                 # 테스트 세트 크기: 전체의 20%
validation_size = int(total_size * validation_ratio)     # 검증 세트 크기: 전체의 20%
train_size = total_size - test_size - validation_size    # 훈련 세트 크기: 전체의 60%

`np.random.permutation()` 함수를 이용하여 인덱스를 무작위로 섞는다. 

In [ ]:
rnd_indices = np.random.permutation(total_size)

훈련 세트, 검증 세트, 테스트 세트를 각 6:2:2 비율로 나눈다.

In [ ]:
X_train = X_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]

X_valid = X_with_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]

X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]

### __단계 3: 타깃 변환__ 



타깃은 0, 1로 설정되어 있다. 타겟으로 설정한 세토사는 1로, 나머지 버시컬러, 버지니카는 0으로 나온다.
훈련 세트의 첫 5개 샘플의 품종은 다음과 같다.

In [ ]:
y_train[:5]

array([0, 0, 1, 0, 0])

원-핫 인코딩 사용


In [ ]:
def to_one_hot(y):
    n_classes = y.max() + 1                 # 클래스 수
    m = len(y)                              # 샘플 수
    Y_one_hot = np.zeros((m, n_classes))    # (샘플 수, 클래스 수) 0-벡터 생성
    Y_one_hot[np.arange(m), y] = 1          # 샘플 별로 해당 클래스의 값만 1로 변경. (넘파이 인덱싱 활용)
    return Y_one_hot

샘플 5개에 대해 잘 작동하는 것을 확인할 수 있다.

In [ ]:
y_train[:5]

array([0, 0, 1, 0, 0])

In [ ]:
to_one_hot(y_train[:5])

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.]])

이제 훈련/검증/테스트 세트의 타깃을 모두 원-핫 벡터로 변환한다.

In [ ]:
Y_train_one_hot = to_one_hot(y_train)
Y_valid_one_hot = to_one_hot(y_valid)
Y_test_one_hot = to_one_hot(y_test)

### __단계 4: 로지스틱 함수 구현__ 



아래처럼 정의된 로지스틱 함수를 파이썬 함수로 구현한다. 

$$
\sigma(\mathbf{t})
= \dfrac{1}{{1}+{e}^{-t}}
$$


In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

### __단계 5: 경사하강법 활용 훈련__ 

소프트맥스의 비용함수에서 k가 2일 경우 로지스틱 회귀의 비용함수와 같기때문에 수정하지 않고 그대로 사용하였다.

이제 훈련 코드를 작성할 수 있다.
클래스별 파라미터로 이루어진 $(n+1, K)$ 행렬 모양의 2차원 넘파이 어레이 $\Theta$를 
생성하기 위해 $n$과 $K$를 확인한다.

In [ ]:
n_inputs = X_train.shape[1]           # 특성 수(n) + 1, 붓꽃의 경우: 특성 2개 + 1
n_outputs = len(np.unique(y_train))   # 중복을 제거한 클래스 수(K), 붓꽃의 경우: 3개

In [ ]:
n_inputs

2

In [ ]:
n_outputs

2

파라미터 $\Theta$를 무작위로 초기 설정한다.

In [ ]:
Theta = np.random.randn(n_inputs, n_outputs)

배치 경사하강법 훈련은 아래 코드를 통해 이루어진다.

- `eta = 0.01`: 학습률
- `n_iterations = 5001` : 에포크 수
- `m = len(X_train)`: 훈련 세트 크기, 즉 훈련 샘플 수
- `epsilon = 1e-7`: $\log$ 값이 항상 계산되도록 더해지는 작은 실수
- `logits`: 모든 샘플에 대한 클래스별 점수, 즉 $\mathbf{X}_{\textit{train}}\, \Theta$
- `Y_proba`: 모든 샘플에 대해 계산된 이진분류 확률, 즉 $\hat P$

In [ ]:
#  배치 경사하강법 구현
eta = 0.01
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7

for iteration in range(n_iterations):     # 5001번 반복 훈련
    logits = X_train.dot(Theta)
    Y_proba = sigmoid(logits)
    
    if iteration % 500 == 0:              # 500 에포크마다 손실(비용) 계산해서 출력
        loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
        print(iteration, loss)
    
    error = Y_proba - Y_train_one_hot     # 그레이디언트 계산.
    gradients = 1/m * X_train.T.dot(error)
    
    Theta = Theta - eta * gradients       # 파라미터 업데이트

0 0.6216585388989472
500 0.5358155407854331
1000 0.46868915858417054
1500 0.4228110723139094
2000 0.389997119276909
2500 0.36532453055572167
3000 0.34600634309210915
3500 0.33038385603797926
4000 0.3174181616594949
4500 0.3064286658965357
5000 0.2969519170535297


학습된 파라미터는 다음과 같다.

In [ ]:
Theta

array([[ 3.49367474, -3.54330311],
       [-2.15880998,  2.18878276]])

검증 세트에 대한 예측과 정확도는 다음과 같다.
`logits`, `Y_proba`를 검증 세트인 `X_valid`를 이용하여 계산한다.
예측 클래스는 `Y_proba`에서 가장 큰 값을 갖는 인덱스로 선택한다.

In [ ]:
logits = X_valid.dot(Theta)              
Y_proba = sigmoid(logits)
y_predict = np.argmax(Y_proba, axis=1)          # 가장 높은 확률을 갖는 클래스 선택

accuracy_score = np.mean(y_predict == y_valid)  # 정확도 계산
accuracy_score

0.9666666666666667

### __단계 6: 규제가 추가된 경사하강법 활용 훈련__ 

alpha = 규제 하이퍼파라미터

In [ ]:
eta = 0.1
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.1   # 규제 하이퍼파라미터

Theta = np.random.randn(n_inputs, n_outputs)  # 파라미터 새로 초기화

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = sigmoid(logits)
    
    if iteration % 500 == 0:
        xentropy_loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
        l2_loss = 1/2 * np.sum(np.square(Theta[1:]))  # 편향은 규제에서 제외
        loss = xentropy_loss + alpha * l2_loss        # l2 규제가 추가된 손실
        print(iteration, loss)
    
    error = Y_proba - Y_train_one_hot
    l2_loss_gradients = np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]   # l2 규제 그레이디언트
    gradients = 1/m * X_train.T.dot(error) + l2_loss_gradients
    
    Theta = Theta - eta * gradients

0 0.7162610260856582
500 0.5424422902989774
1000 0.5489288755349917
1500 0.549640024455795
2000 0.5497072521674562
2500 0.5497135348935831
3000 0.5497141214229124
3500 0.5497141761734805
4000 0.5497141812842178
4500 0.5497141817612832
5000 0.5497141818058153


In [ ]:
logits = X_valid.dot(Theta)
Y_proba = sigmoid(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9

규제를 적용하니 정확도가 더 떨어졌다

규제를 적용하기 전보다 정확도가 떨어졌으니 규제를 약화시켜야 한다.

### __단계 7: 조기 종료 추가__

위 규제가 사용된 모델의 훈련 과정에서
매 에포크마다 검증 세트에 대한 손실을 계산하여 오차가 줄어들다가 증가하기 시작할 때 멈추도록 한다.

이전 단계에서 규제를 적용하였을때 정확도가 떨어졌으니 규제를 0.01로 약하게 정하였다.

In [ ]:
eta = 0.1 
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.01              # 규제 하이퍼파라미터
best_loss = np.infty   # 최소 손실값 기억 변수

Theta = np.random.randn(n_inputs, n_outputs)  # 파라미터 새로 초기화

for iteration in range(n_iterations):
    # 훈련 및 손실 계산
    logits = X_train.dot(Theta)
    Y_proba = sigmoid(logits)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]
    Theta = Theta - eta * gradients

    # 검증 세트에 대한 손실 계산
    logits = X_valid.dot(Theta)
    Y_proba = sigmoid(logits)
    xentropy_loss = -np.mean(np.sum(Y_valid_one_hot * np.log(Y_proba + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
    loss = xentropy_loss + alpha * l2_loss
    
    # 500 에포크마다 검증 세트에 대한 손실 출력
    if iteration % 500 == 0:
        print(iteration, loss)
        
    # 에포크마다 최소 손실값 업데이트
    if loss < best_loss:
        best_loss = loss
    else:                                      # 에포크가 줄어들지 않으면 바로 훈련 종료
        print(iteration - 1, best_loss)        # 종료되지 이전 에포크의 손실값 출력
        print(iteration, loss, "조기 종료!")
        break

0 0.7456585550574103
500 0.3388041580108214
1000 0.3201659340346895
1222 0.3193338407410206
1223 0.31933385042637197 조기 종료!


In [ ]:
logits = X_valid.dot(Theta)
Y_proba = sigmoid(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9666666666666667

규제 적용 전보다 정확도가 더 올라간것을 볼 수 있다.

## 과제 2

과제 1에서 구현된 로지스틱 회귀 알고리즘에 일대다(OvR) 방식을 적용하여 붓꽃에 대한 다중 클래스 분류 알고리즘을 구현하라. 단, 사이킷런을 전혀 사용하지 않아야 한다.

## 과제 3

A. 사진을 낮과 밤으로 분류하는 로지스틱 회귀 모델을 구현하라.

B. 사진을 낮과 밤, 실내와 실외로 분류하는 다중 레이블 분류 모델을 두 개의 로지스틱 회귀 모델을 이용하여 구현하라.

단, 모델 구현에 필요한 사진을 직접 구해야 한다. 최소 100장 이상의 사진 활용할 것.